In [2]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00


In [3]:
import os
import pandas as pd
from datasets import load_dataset
from sklearn.model_selection import train_test_split

In [4]:
# df = pd.read_excel('/content/consolidated_labelling_data (2).xlsx')
df = pd.read_csv('/content/combined_data_cleaned_without_prompt_new4.csv')

In [5]:
df.head()

,text,cleaned_text,sentiment
0,I am sure Bacha Khan in heaven will cry on the...,I am sure Bacha Khan in heaven will cry on the...,1
1,"Radicals', 'Religion of Peace;!!! dont use isl...","Radicals, 'Religion of Peace;!!! dont use isla...",0
2,He was willing to shoot an innocent man but co...,He was willing to shoot an innocent man but co...,1
3,No Christian are save in Pakistan,No Christian are save in Pakistan.,-1
4,You should recognize that justice and fairness...,You should recognize that justice and fairness...,1


In [6]:
df.shape

(2357, 3)

In [ ]:
# ! pip install -q langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993227 sha256=8d43d659d6cb00a04e04be952a5a37118d8741ca6e867d8647d0bc6431be3f52
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
from langdetect import detect

# Function to detect language
def detect_language(text):
    try:
        lang = detect(text)
        return lang == 'en'
    except:
        return False  # Handle cases where language detection fails

# Apply language detection function to filter English records
df = df[df['Comments'].apply(detect_language)]
df.shape

(1894, 5)

In [ ]:
# def clean_and_concat(text):
#     # Remove specified characters and replace with whitespace
#     cleaned_text = text.replace('\n', ' ').replace('*', '').replace('@', '').replace('#', '').replace("'", '').replace('"', '')
#     # Split text by lines, remove empty lines, and join lines with single whitespace
#     cleaned_text = ' '.join([line.strip() for line in cleaned_text.split('\n') if line.strip()])
#     return cleaned_text

# # Apply function to clean and concatenate text in each row
# df['Comments'] = df['Comments'].apply(clean_and_concat)

<ipython-input-11-3e4f15677f3f>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Comments'] = df['Comments'].apply(clean_and_concat)


In [ ]:
# df = df.rename(columns={2: 'labels', 3: 'text'})

In [ ]:
# df.head()

In [7]:
df.isnull().sum()

text            0
cleaned_text    2
sentiment       0
dtype: int64

In [8]:
df["sentiment"].value_counts()

sentiment
 1    1436
-1     601
 0     320
Name: count, dtype: int64

In [9]:
df[df['cleaned_text'].isnull()]

,text,cleaned_text,sentiment
579,#YaAllah_One_And_Mumtaz_Qadri_Ki_Pakistan #nee...,NaN,-1
1727,#ExecuteBlasphemers\n#OurProphetOurHonour,NaN,-1


In [10]:
df.dropna(subset=["cleaned_text"], inplace=True)

In [ ]:
# df = df[df["labels"] != 'Irrelevant']

In [11]:
# check null values
print(df.isnull().sum())

text            0
cleaned_text    0
sentiment       0
dtype: int64


In [ ]:
# df = df[:200]
# df.shape

In [13]:
df['sentiment'].unique()

array([ 1,  0, -1])

In [14]:
type(df['sentiment'][0])

numpy.int64

In [ ]:
# df['Sentiment'] = df['Sentiment'].astype('int64')
# type(df['Sentiment'][0])

numpy.int64

In [15]:
df.head()

,text,cleaned_text,sentiment
0,I am sure Bacha Khan in heaven will cry on the...,I am sure Bacha Khan in heaven will cry on the...,1
1,"Radicals', 'Religion of Peace;!!! dont use isl...","Radicals, 'Religion of Peace;!!! dont use isla...",0
2,He was willing to shoot an innocent man but co...,He was willing to shoot an innocent man but co...,1
3,No Christian are save in Pakistan,No Christian are save in Pakistan.,-1
4,You should recognize that justice and fairness...,You should recognize that justice and fairness...,1


In [ ]:
# df.to_csv("Consolidated_labelling_data_preprocessed.csv", index=False)

In [16]:
df = df.drop(columns=["text"])

In [17]:
df.rename(columns={'cleaned_text': 'text', 'sentiment': 'label'}, inplace=True)
df.head()

,text,label
0,I am sure Bacha Khan in heaven will cry on the...,1
1,"Radicals, 'Religion of Peace;!!! dont use isla...",0
2,He was willing to shoot an innocent man but co...,1
3,No Christian are save in Pakistan.,-1
4,You should recognize that justice and fairness...,1


In [18]:
type(df['text'][0])

str

In [19]:
# Split the train data => {train, eval, test}

train, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label']) #(80-20)
eval, test = train_test_split(test_df, test_size=0.5, random_state=42, stratify=test_df['label']) #(50 - 50)

In [20]:
print(f"new dataframe shapes: train is {train.shape}, eval is {eval.shape}")

new dataframe shapes: train is (1884, 2), eval is (235, 2)


In [21]:
train.to_csv("train_subset.csv", index=False)
eval.to_csv("eval_subset.csv", index=False)

In [22]:
dataset = load_dataset('csv',
                        data_files={'train': 'train_subset.csv',
                        'eval': 'eval_subset.csv'}, encoding = "ISO-8859-1")
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1884
    })
    eval: Dataset({
        features: ['text', 'label'],
        num_rows: 235
    })
})

In [23]:
#create a function to convert label
def transform_labels(label):

    label = label['label']
    num = 0
    if label == -1:
        num = 0
    elif label == 0:
        num = 1
    elif label == 1:
        num = 2

    return {'label': num}


# Transform labels and remove the useless columns
remove_columns = ['label']
dataset = dataset.map(transform_labels, remove_columns=remove_columns)
dataset

Map:   0%|          | 0/1884 [00:00<?, ? examples/s]

Map:   0%|          | 0/235 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1884
    })
    eval: Dataset({
        features: ['text', 'label'],
        num_rows: 235
    })
})

In [24]:
!pip install -q transformers simpletransformers nltk transformers[torch]
#!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.3 MB/s eta 0:00:00


In [25]:
from transformers import AutoTokenizer

model_name = "bert-base-uncased"
# model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"

tokenizer = AutoTokenizer.from_pretrained(model_name)

# let's tokenize the data for the model to be able to understand
def tokenize_data(example):
    return tokenizer(example["text"], padding='max_length', truncation=True)

# Tokenize the dataset
dataset = dataset.map(tokenize_data, batched=True)
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/1884 [00:00<?, ? examples/s]

Map:   0%|          | 0/235 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1884
    })
    eval: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 235
    })
})

In [26]:
from transformers import AutoModelForSequenceClassification
# from transformers import TFRobertaForSequenceClassification

# Loading a pretrain model while specifying the number of labels in our dataset for fine-tuning
num_labels = 3
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# model = TFRobertaForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
df.head()

,text,label
0,I am sure Bacha Khan in heaven will cry on the...,1
1,"Radicals, 'Religion of Peace;!!! dont use isla...",0
2,He was willing to shoot an innocent man but co...,1
3,No Christian are save in Pakistan.,-1
4,You should recognize that justice and fairness...,1


In [28]:
# let's set the training arguements
from transformers import TrainingArguments

# the default batch size for training arguments
batch_size = 8

# set number of epochs
number_of_epochs = 3 #7
# let set the logging steps
logging_steps = len(dataset['train']) // batch_size # it should log each batch

steps = (len(dataset['train']) / batch_size) * number_of_epochs
warmup_steps = int(0.2 * steps)


# training_args = TrainingArguments(
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     num_train_epochs=3, #number_of_epochs
#     logging_dir='./logs',
#     logging_steps=100, #logging_steps
#     save_steps=1000,
#     evaluation_strategy='epoch', #steps
#     save_total_limit=3,
#     output_dir="fine-tuned-twitter-roberta-base"
# )



training_args = TrainingArguments(
                                  num_train_epochs=number_of_epochs,
                                  load_best_model_at_end=True,
                                  evaluation_strategy='steps',
                                  save_strategy='steps',
                                  learning_rate=2e-5,
                                  logging_steps=logging_steps,
                                  warmup_steps= warmup_steps,
                                  save_steps=1000,
                                  eval_steps=500,
                                  output_dir="fine-tuned-bert-base-uncased"
                                  )

In [29]:
# shuffle the datasets

train_dataset = dataset['train'].shuffle(seed=10)
eval_dataset = dataset['eval'].shuffle(seed=10)

In [30]:
from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset
)

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     tokenizer=tokenizer,
#     train_dataset=train_dataset,
#     eval_dataset=eval_dataset,
# )

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [31]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1884
    })
    eval: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 235
    })
})

In [ ]:
# columns_to_remove = ['input_ids', 'attention_mask']

# # Remove the specified columns
# dataset = dataset.remove_columns(columns_to_remove)

In [32]:
# Launch the learning process: training
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: ERROR API key must be 40 characters long, yours was 37


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss
500,0.773400,0.890907


TrainOutput(global_step=708, training_loss=0.7502523951948026, metrics={'train_runtime': 639.3655, 'train_samples_per_second': 8.84, 'train_steps_per_second': 1.107, 'total_flos': 1487117036998656.0, 'train_loss': 0.7502523951948026, 'epoch': 3.0})

In [33]:
import numpy as np
from sklearn.metrics import mean_squared_error
from datasets import load_metric


def compute_metrics(eval_pred):
    # load the metrics to use
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # calculate the mertic using the predicted and true value
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)
    f1 = load_f1.compute(predictions=predictions, references=labels, average="weighted")
    return {"accuracy": accuracy, "f1score": f1}

In [34]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,

)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [35]:
# Launch the final evaluation
trainer.evaluate()

<ipython-input-33-329e5f56ea92>:8: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Trainer is attempting to log a value of "{'accuracy': 0.6638297872340425}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.6422138281242185}" of type <class 'dict'> for key "eval/f1score" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.9401269555091858,
 'eval_accuracy': {'accuracy': 0.6638297872340425},
 'eval_f1score': {'f1': 0.6422138281242185},
 'eval_runtime': 9.5915,
 'eval_samples_per_second': 24.501,
 'eval_steps_per_second': 3.128}

In [ ]:
# df_test = pd.read_csv('/content/twitter_validation.csv', header=None)

In [ ]:
# df_test = df_test.rename(columns={2: 'labels', 3: 'text'})
# df_test.dropna(subset=["text"], inplace=True)
# df_test = df_test.drop(columns=[0, 1])
# df_test = df_test[df_test["labels"] != 'Irrelevant']
# df_test.head()

In [36]:
test.to_csv("test_subset.csv", index=False)

In [37]:
dataset_test = load_dataset('csv',
                        data_files={'test': 'test_subset.csv'}, encoding = "ISO-8859-1")

Generating test split: 0 examples [00:00, ? examples/s]

In [38]:
dataset_test

DatasetDict({
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 236
    })
})

In [39]:
# Transform labels and remove the useless columns
remove_columns = ['label']
dataset_test = dataset_test.map(transform_labels, remove_columns=remove_columns)

Map:   0%|          | 0/236 [00:00<?, ? examples/s]

In [40]:
dataset_test = dataset_test.map(tokenize_data, batched=True)

Map:   0%|          | 0/236 [00:00<?, ? examples/s]

In [41]:
test_dataset = dataset_test['test'].shuffle(seed=10)

In [42]:
results = trainer.evaluate(test_dataset)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next majo

In [43]:
output_dir = "model"

# Save the model
trainer.save_model(output_dir)

In [44]:
import shutil

output_dir = "model"

# Path for the zip file
zip_file_path = "finetuned_model"

# Compress the directory
shutil.make_archive(zip_file_path, 'zip', output_dir)

'/content/finetuned_model.zip'